# Selenium DOC dashboard scraper

THis notebook documents and executes the collection of daily COVID-19 data from the Pennsylvania DOC COVID-19 dashboard. Data is scraped from the DOC dashboard using Selenium and is then processed to conform to a standard dataframe format. 

Data fields collected in this notebook are only for incarcerated people (page 5 of the DOC dashboard), including positive tests, negative tests, pending tests, recovered cases and incarcerated person deaths. 


### Data collection procedure:
1. route selenium driver to DOC dashboard url
2. navigate driver to page 5 of dashboard
3. select date and SCI from dropdown menus
4. right click pie-chart to make table
5. isolate and scrape relevant data fields
6. format and store relevant data fields

In [26]:
# import libraries

import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib import request
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

import re

### 1. route selenium driver to DOC dashboard URL

In [99]:
dash_url = "https://app.powerbigov.us/view?r=eyJrIjoiNWQ5YTQ4ZWUtY2NjMi00ZWRhLTgyNWQtYzAzNzc5NmYwMGIyIiwidCI6IjQxOGUyODQxLTAxMjgtNGRkNS05YjZjLTQ3ZmM1YTlhMWJkZSJ9"
driver = webdriver.Safari()
driver.get(dash_url)

### 2. navigate driver to page 5 of dashboard

In [100]:
for i in range(4):
    driver.execute_script("document.querySelector('[title=\"Next Page\"]').click()")


### 4. right click pie-chart to make table

In [101]:
action = ActionChains(driver)
action.context_click(driver.find_element_by_class_name('donutChart')).perform()
driver.execute_script("document.querySelector('[title=\"Show as a table\"]').click()")

### 5. isolate and scrape relevant data fields

In [115]:
header_div = driver.find_element_by_class_name('columnHeaders')
headers = [header.text for header in header_div.find_elements_by_class_name('pivotTableCellWrap')]

value_div = driver.find_element_by_class_name('bodyCells')
values = [value.text for value in value_div.find_elements_by_class_name('pivotTableCellWrap')]